# Project 2

## Load libs

In [5]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Function
import torch.optim as optim

## Model definitions

In [169]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[-1] # Only keep the last output in the sequence
        out = self.fc(lstm_out)
        return out

## Train, val, test loops

In [170]:
from tqdm.notebook import tqdm # status bar

In [171]:
def train(model, data, loss_fn, optimizer, epochs=3):

    for epoch in range(epochs):

        epoch_loss = []

        for batch, (samples, labels) in enumerate(tqdm(data)):

            # forward pass
            prediction = model(samples)
            loss = loss_fn(prediction, labels)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # record loss
            epoch_loss.append(loss.log10().item())
        
        # Print the loss for this epoch
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
    return epoch_loss

## Load data

In [172]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [187]:
class BikeDataset(Dataset):
    def __init__(self, csv_file, seq_length):
        self.data_frame = pd.read_csv(csv_file)
        self.seq_length = seq_length
        
    def __len__(self):
        return len(self.data_frame) - self.seq_length
    
    def __getitem__(self, index):
        beg_idx = index
        end_idx = index+self.seq_length
        input_features = torch.tensor(self.data_frame.iloc[beg_idx:end_idx,:-1].values,dtype=torch.float32)
        target_label = torch.tensor(self.data_frame.iloc[end_idx,-1],dtype=torch.float32)
        #row = self.data_frame.iloc[index]
        #input_features = row.iloc[:-1].values.astype(float)
        #input_features = torch.tensor(input_features, dtype=torch.float32)
        #target_label = row.iloc[-1].values.astype(float)
        #target_label = torch.tensor(target_label, dtype=torch.float32)
        return input_features, target_label

In [188]:
# Define the CSV file path
csv_file = 'data/Bike-Sharing-Dataset/hour.csv'

# Create an instance of the custom dataset
dataset = BikeDataset(csv_file, 6)

In [189]:
dataset.data_frame.iloc[0:4,:-1].shape

(4, 11)

In [190]:
# Use PyTorch's DataLoader to create a data loader for batching and shuffling
batch_size = 32
shuffle = True
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# Iterate over the data loader to access batches of data
for batch in data_loader:
    input_features, target_label = batch
#     print('Input Features:', input_features)
#     print('Target Label:', target_label)
    print(input_features.shape)
    print(target_label.shape)

torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 

torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 

torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 6, 11])
torch.Size([32])
torch.Size([32, 

## Fit models

### LSTM

In [ ]:
# Define the input dimensions, hidden dimensions, and output dimensions
input_dim = 11
hidden_dim = 32
output_dim = 1

# Create an instance of the LSTM model
model = LSTMModel(input_dim, hidden_dim, output_dim)

# Define the loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
_ = train(model, data_loader, loss_fn, optimizer, epochs=2)

In [119]:
# Train the LSTM model
num_epochs = 1000
for epoch in range(num_epochs):
    
    # Forward pass
    output = model(data)
    loss = loss_fn(output, data[-1].view(1, -1))
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print the loss for this epoch
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

AttributeError: module 'torch.utils.data' has no attribute 'dim'

In [133]:
# Evaluate the LSTM model
with torch.no_grad():
    model.eval()
    test_input = torch.randn(5, input_dim).unsqueeze(1)
    test_output = model(test_input)
    print('Input:', test_input.view(-1).numpy())
    print('Output:', test_output.view(-1).numpy())

Input: [ 0.40316468 -1.6099793  -0.18292025  0.7831274   0.56583905  0.8367732
  0.13318895  0.10420565  1.4081478   0.254313    0.5925332  -0.39444527
 -2.35443     2.346701    0.34729904  0.5855038  -0.21909761 -1.2861074
 -0.13845819  0.95865726 -0.63061947  0.19785738 -0.56747246  1.1754051
 -0.10281649  1.0431526   0.73668116  0.9788994   0.9719     -0.13420518
  1.868313   -1.6401696  -0.89826924 -0.18196972  0.66872144 -0.47356382
  1.1221837  -0.35528487  0.75093406 -1.9764977   0.15223454 -0.9766444
  1.7834507  -0.5267382  -0.9601347   2.6272225   0.9146143   0.4126982
 -1.5323913  -0.84161323  0.6803284  -0.29723397 -1.5426215   2.5198333
  0.95400983]
Output: [73.85306]


## Results